In [ ]:
import os
import numpy as np
import pandas as pd
import random
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import seaborn as sns
from contextlib import contextmanager
from time import time
from tqdm import tqdm
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report, log_loss, accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from sklearn.mixture import GaussianMixture, BayesianGaussianMixture
from sklearn.metrics import accuracy_score

In [ ]:
#  Data preparation

In [ ]:
data0=pd.read_csv('../input/online-sales-in-usa/sales.csv')
display(data0[0:5].T)
print(data0.columns.tolist())

In [ ]:
dataA=data0[['User Name','Gender','age']].drop_duplicates()
dataA=dataA.reset_index(drop=True)
display(dataA)

In [ ]:
print(data0['category'].unique().tolist())

In [ ]:
dataC=data0[['User Name','category']]
dataC['count']=1
dataC2=dataC.groupby(['User Name','category'],as_index=False).sum()
display(dataC2)

In [ ]:
dataD=dataC2.set_index(['User Name','category']).unstack()['count']
dataD=dataD.fillna(0)
display(dataD)

In [ ]:
dataD2=dataD.copy()
for i in range(len(dataD)):
    dataD2.iloc[i,:]=dataD.iloc[i,:]/dataD.iloc[i,:].sum()
display(dataD2)

In [ ]:
bgm = BayesianGaussianMixture(
    n_components=7,
    max_iter=300,
    n_init=10,
    random_state=2,
    verbose_interval=100,
)

In [ ]:
bgm_labels = bgm.fit_predict(dataD2)
dataD2['label']=bgm_labels
display(dataD2)
display(dataD2['label'].value_counts())

In [ ]:
dataD3=dataD2.groupby('label').mean()
display(dataD3)

In [ ]:
fig=make_subplots(specs=[[{"secondary_y":False}]])
for i in range(7):
    fig.add_trace(go.Scatter(x=dataD3.columns.tolist(),y=dataD3.iloc[i],name='label='+str(i)),secondary_y=False,)

fig.update_layout(autosize=False,width=900,height=500,title_text="Online Sales Customers Clustering")
fig.update_xaxes(title_text="Category")
fig.update_yaxes(title_text="Order Ratio",secondary_y=False)
fig.show()